In [2]:
import pandas as pd
import scanpy as sc
from os.path import join
import numpy as np
import anndata
from utils import *
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
import pickle
import os
import warnings
from itertools import combinations
warnings.filterwarnings("ignore")

In [3]:
import utils

In [4]:
means = utils.get_means()

True /storage/groups/ml01/workspace/mo/for_nacho/Norman2019_prep_new.h5ad
True /storage/groups/ml01/workspace/carlo.dedonno/cpa-reproducibility/fig4_predicted_adata_full.h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


True /storage/groups/ml01/workspace/carlo.dedonno/cpa-reproducibility/figure4_latent_adata.h5ad

dimensions of embedding: (497800, 64)
all good. Removing datasets based on repetition is then tractable


In [34]:
print('# uniq combinations')
uniq = set([x for lab in ['a', 'b'] for x in means.obs[lab] if str(x) != 'nan'])
print(uniq)
import itertools
print(len(list(itertools.combinations(uniq, r=2))))

# uniq combinations
{'CKS1B', 'ELMSAN1', 'TGFBR2', 'SLC6A9', 'DLX2', 'RUNX1T1', 'GLB1L2', 'HES7', 'KMT2A', 'KIAA1804', 'MAPK1', 'CBL', 'CNNM4', 'MAP7D1', 'IKZF3', 'ETS2', 'FOXL2', 'CEBPA', 'OSR2', 'ctrl', 'DUSP9', 'PTPN13', 'TBX3', 'HNF4A', 'FOSB', 'POU3F2', 'CEBPE', 'CELF2', 'COL2A1', 'ZC3HAV1', 'MAP4K5', 'NCL', 'RHOXF2', 'IGDCC3', 'NIT1', 'ARID1A', 'STIL', 'KIF18B', 'JUN', 'AHR', 'CLDN6', 'C19orf26', 'CDKN1C', 'CNN1', 'HK2', 'CDKN1B', 'ZBTB25', 'BCORL1', 'IRF1', 'MEIS1', 'PTPN1', 'SLC38A2', 'BPGM', 'UBASH3A', 'ZBTB1', 'ZNF318', 'ARRDC3', 'ISL2', 'CDKN1A', 'PRDM1', 'SLC4A1', 'ATL1', 'HOXA13', 'BCL2L11', 'PTPN12', 'MIDN', 'PTPN9', 'UBASH3B', 'TSC22D1', 'SPI1', 'TBX2', 'SNAI1', 'KIF2C', 'HOXB9', 'SET', 'COL1A1', 'IER5L', 'RREB1', 'BAK1', 'PLK4', 'CBFA2T3', 'MAP2K6', 'EGR1', 'MAP2K3', 'ZBTB10', 'SAMD1', 'MAP4K3', 'FOXO4', 'MAML2', 'TMSB4X', 'CSRNP1', 'CEBPB', 'FOXF1', 'LYL1', 'FEV', 'LHX1', 'FOXA1', 'HOXC13', 'CITED1', 'TP73', 'S1PR2', 'KLF1', 'PRTG', 'FOXA3', 'SGK1', 'C3orf72'}
5565


In [213]:
queries = [c for c in combinations([s for s in set(means.obs[['a', 'b']].values.flatten()) if str(s) != 'nan'], r=2)]

In [214]:
len(queries)

5565

In [11]:
# means.obs.condition.value_counts()

In [50]:

# def calculate_rf_multithread(a, b):
#     # gene_names = set(ori.obs[['a', 'b']].values.flatten())
#     print(a, b)

#     if str(a) == 'nan' or a == 'ctrl':
#         return
#     if str(b) == 'nan' or b == 'ctrl':
#         return
#     bkp_path = '../data/rf_genes/%s_%s.pkl' % (a, b)
#     print(os.path.exists(bkp_path), bkp_path)
#     if os.path.exists(bkp_path):
#         return
#     try:
#         print(a, b)
#         ctrl = means[means.obs['condition'] == 'ctrl']
#         gene_name = a
#         lab_cond_a = {'%s+ctrl' % a, 'ctrl+%s' % a}
#         lab_cond_b = {'%s+ctrl' % b, 'ctrl+%s' % b}
#         cond_single_a = means[means.obs['condition'].isin(lab_cond_a),:]
#         cond_single_b = means[means.obs['condition'].isin(lab_cond_b),:]

#         lab_cond_double = {'%s+%s' % (a, b), '%s+%s' % (b, a)}
#         cond_double_ab_ori = means[means.obs['condition'].isin(lab_cond_double),:]
#         cond_double_ab_pred = means[means.obs['condition'].isin(lab_cond_double),:]

#         print(cond_double_ab_ori.shape, cond_double_ab_pred.shape)
#         ctrl.obs['location'] = 'control'
#         cond_single_a.obs['location'] = 'a'
#         cond_single_b.obs['location'] = 'b'
#         cond_double_ab_ori.obs['location'] = 'ab'
#         cond_double_ab_pred.obs['location'] = 'ab'

#         cond_double_sel = cond_double_ab_ori if cond_double_ab_ori.shape[0] != 0 else cond_double_ab_pred if cond_double_ab_pred.shape[0] != 0 else None

#         if cond_double_sel is None:
#             return
#         assert cond_double_sel is not None

#         print(ctrl.shape, cond_single_a.shape, cond_single_b.shape, cond_double_sel.shape)

#         # this will throw an error sometimes because the var_names of ori and pred are not the same
#         ad2 = ctrl.concatenate([cond_single_a, cond_single_b, cond_double_sel])
#         # ad = ad.concatenate(cond_double)

#         print(list(ad2.obs['location'].value_counts()))

#         # print(ad2.shape)

#         X, y = np.array(ad2.to_df()), np.array(ad2.obs['location'])
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=500, stratify=y)

#         clf_tree = RandomForestClassifier(class_weight='balanced')

#         # print('fitting RF....')
#         clf_tree.fit(X_train, y_train)

#         y_predict = clf_tree.predict(X_test)
#         acc = accuracy_score(y_test, y_predict)
#         # print('Feature prediction accuracy (test size: {1:.1f}%): {0}\n'.format(acc, 100*.2))

#         X, y = np.array(ad2.to_df()), np.array(ad2.obs['location'])
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=500, stratify=y)

#         clf_tree = RandomForestClassifier(class_weight='balanced')
#         clf_tree.fit(X_train, y_train)

#         y_predict = clf_tree.predict(X_test)
#         acc = accuracy_score(y_test, y_predict)
#         print('Feature prediction accuracy (test size: {1:.1f}%): {0}\n'.format(acc, 100*.2))

#         importances = clf_tree.feature_importances_
#         importances = pd.Series(importances, index=ad2.var.index).sort_values(ascending=False)

#         print('saving object')
#         pickle.dump(importances, open(bkp_path, 'wb'))        
#     except IOError:
#         print('error with gene', a, b)
        

        

In [206]:
uniq_genes = set(means.obs[['a', 'b']].values.flatten())

In [216]:
i = 0
for g1, g2 in itertools.combinations(uniq_genes, r=2):
    if str(g1) == 'nan':
        continue
    if str(g2) == 'nan':
        continue

    i += 1
    
    # if not 'KLF1' in g1 and not 'KLF1' in g2:
    #     continue
    # if not 'FOXA3' in g1 and not 'FOXA3' in g2:
    #     continue
    # print(g1, g2)    
    
    if i % 100 == 0:
        print(i, g1, g2)
    g1_mask = ((means.obs['a'] == g1) & means.obs['b'].str.contains('ctrl')) | ((means.obs['b'] == g1) & means.obs['a'].str.contains('ctrl'))
    g2_mask = ((means.obs['a'] == g2) & means.obs['b'].str.contains('ctrl')) | ((means.obs['b'] == g2) & means.obs['a'].str.contains('ctrl'))
    g12_mask = ((means.obs['a'] == g1) & (means.obs['b'] == g2)) | ((means.obs['a'] == g2) & (means.obs['b'] == g1))
    g0_mask = (means.obs.index == 'ctrl')

    ab = means[g12_mask.values,:] 
    a = means[g1_mask.values,:]
    b = means[g2_mask.values,:]
    ctrl = means[g0_mask,:]

    outpath = '../../data/rf_genes/%s_%s.tsv.gz' % (g1, g2)
    if os.path.exists(outpath):
        continue
    # outpath2 = '../../data/rf_genes/%s_%s.pkl' % g2, g1
    
    # print(ab.shape, a.shape, b.shape, ctrl.shape)
    # b = means[means.obs['a'].str.contains(g2) | means.obs['b'].str.contains('ctrl'),:]

    pred = pd.concat([ab.to_df(), a.to_df(), b.to_df(), ctrl.to_df()])
    top_100 = pred.var(axis=0).sort_values()[::-1][:100].index
    for c in pred:
        pred[c] = (pred[c] - np.mean(pred[c])) / np.std(pred[c])
    pred = pred.T.reindex(top_100).T
    pred = pred[pred.index != 'ctrl']
    pred = pred.T

    pred_cols = []
    for c in pred:
        # print(c)
        a, b = c.split('+')
        c = (a + '+' + b) if (a > b or a == 'ctrl') else (b + '+' + a)
        pred_cols.append(c)
    pred.columns = pred_cols
    pred = pred.T
    pred = pred.groupby(pred.index).mean()
    
    # print(pred.shape)
    pd.Series(pred.T.index).to_csv(outpath)
    # break

100 CKS1B S1PR2
200 ELMSAN1 FOXA1
300 TGFBR2 LYL1
400 SLC6A9 CEBPB
500 DLX2 CSRNP1
600 RUNX1T1 CSRNP1
700 GLB1L2 CEBPB
800 HES7 LYL1
900 KMT2A FOXA1
1000 MAPK1 S1PR2
1100 KIAA1804 C3orf72
1200 CNNM4 OSR2
1300 MAP7D1 CEBPE
1400 IKZF3 ARID1A
1500 ETS2 CDKN1B
1600 FOXL2 ARRDC3
1700 CEBPA TSC22D1
1800 OSR2 MAP2K6
1900 ctrl LHX1
2000 PTPN13 CEBPE
2100 TBX3 CDKN1C
2200 HNF4A ATL1
2300 FOSB CBFA2T3
2400 POU3F2 S1PR2
2500 CELF2 CDKN1C
2600 COL2A1 PTPN9
2700 ZC3HAV1 CSRNP1
2800 NCL C19orf26
2900 RHOXF2 TSC22D1
3000 IGDCC3 FOXA1
3100 ARID1A ZNF318
3200 STIL MAP4K3
3300 JUN PTPN1
3400 AHR SAMD1
3500 C19orf26 ARRDC3
3600 CNN1 LYL1
3700 HK2 TBX2
3800 ZBTB25 PTPN1
3900 BCORL1 LYL1
4000 MEIS1 CBFA2T3
4100 PTPN1 SNAI1
4200 UBASH3A PTPN9
4300 ZNF318 MIDN
4400 ISL2 TSC22D1
4500 PRDM1 COL1A1
4600 ATL1 MAP4K3
4700 BCL2L11 KLF1
4800 PTPN9 MAP2K6
4900 SPI1 TBX2
5000 SNAI1 KLF1
5100 SET C3orf72
5200 BAK1 CEBPB
5300 EGR1 LYL1
5400 FOXO4 LYL1
5500 FEV LHX1


In [215]:
# pred.shape
# len(queries)
# # queries = [[a, b] for a, b in queries if a == 'KLF1' or b == 'KLF1']
# len(queries)
# from ThreadingUtils import ThreadingUtils
# ThreadingUtils.run(calculate_rf_multithread, queries, n_cores=3)